In [ ]:
# ai_financial_agent.py

import os
import pandas as pd
from PIL import Image
from datetime import datetime

# Vision-Language Model
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Excel Integration
from openpyxl import load_workbook

# Streamlit UI (Optional)
import streamlit as st

# LangChain + LLM Integration
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# LlamaIndex for Document Q&A (optional future extension)
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

# Constants
EXCEL_FILE = "financial_logs.xlsx"
MODEL_NAME = "naver-clova-ix/donut-base"

# Load Donut model
processor = DonutProcessor.from_pretrained(MODEL_NAME)
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)

# Load local LLM
llm = Ollama(model="mistral")

# LangChain Prompt Template
prompt = PromptTemplate(
    input_variables=["records", "question"],
    template="""
You are a financial assistant. Given the following structured records:
{records}

Answer the user's question:
{question}
"""
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

def extract_fields_from_image(image_path: str) -> dict:
    image = Image.open(image_path).convert("RGB")
    task_prompt = "<s>bill"  # TODO: Auto-detect check vs bill

    pixel_values = processor(image, return_tensors="pt").pixel_values
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    outputs = model.generate(pixel_values, decoder_input_ids=decoder_input_ids)
    result = processor.batch_decode(outputs, skip_special_tokens=True)[0]

    # Placeholder parse logic (to be refined)
    fields = {
        "type": "bill",
        "vendor/payee": "Unknown",
        "amount": None,
        "date": None,
        "invoice/check_no": None
    }
    # TODO: Extract fields properly from result text
    return fields

def append_to_excel(data: dict, file_path: str):
    df = pd.DataFrame([data])
    df["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if os.path.exists(file_path):
        book = load_workbook(file_path)
        sheet_name = data["type"] + "s"  # bills or checks
        with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists='overlay') as writer:
            writer.book = book
            if sheet_name not in book.sheetnames:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                start_row = book[sheet_name].max_row
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=start_row)
    else:
        with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
            df.to_excel(writer, sheet_name=data["type"] + "s", index=False)

def validate_file(file) -> bool:
    return file.name.endswith(('.jpg', '.jpeg', '.png', '.pdf'))

def query_financial_data(question: str, excel_path: str) -> str:
    if not os.path.exists(excel_path):
        return "No data available."

    all_data = pd.read_excel(excel_path, sheet_name=None)
    combined_df = pd.concat(all_data.values(), ignore_index=True)
    response = llm_chain.run(records=combined_df.to_csv(index=False), question=question)
    return response

def main():
    st.title("🧾 Financial Document AI Agent")
    uploaded_file = st.file_uploader("Upload a bill or check", type=['jpg', 'jpeg', 'png', 'pdf'])

    if uploaded_file and validate_file(uploaded_file):
        temp_path = f"temp_{uploaded_file.name}"
        with open(temp_path, "wb") as f:
            f.write(uploaded_file.read())

        st.image(temp_path, caption="Uploaded Image")

        with st.spinner("Extracting information..."):
            extracted_data = extract_fields_from_image(temp_path)
            st.json(extracted_data)

        if st.button("Log to Excel"):
            append_to_excel(extracted_data, EXCEL_FILE)
            st.success(f"Logged to {EXCEL_FILE} successfully.")

        os.remove(temp_path)

    st.markdown("---")
    st.subheader("🔎 Ask a Question About Your Financial Logs")
    query = st.text_input("Enter your question (e.g., 'Total amount spent in June'):")
    if query:
        with st.spinner("Analyzing with LLM..."):
            result = query_financial_data(query, EXCEL_FILE)
            st.success(result)

if __name__ == "__main__":
    main()